In [11]:
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.preprocessing.image import ImageDataGenerator

# load dataset
(trainX, trainY), (testX, testY) = mnist.load_data()

In [14]:
# summarize dataset shape
print('Train: ', trainX.shape, trainY.shape)
print('Test: ', (testX.shape, testY.shape))
print('Train:', trainX.min(), trainX.max(), trainX.mean(), trainX.std())
print('Test:', testX.min(), testX.max(), testX.mean(), testX.std())

Train:  (60000, 28, 28) (60000,)
Test:  ((10000, 28, 28), (10000,))
Train: 0 255 33.318421449829934 78.56748998339798
Test: 0 255 33.791224489795916 79.17246322228644


In [16]:
# reshape dataset to have a single channel
width, height, channels = trainX.shape[1], trainX.shape[2], 1
trainX = trainX.reshape((trainX.shape[0], width, height, channels))
testX = testX.reshape((testX.shape[0], width, height, channels))

In [17]:
width, height, channels

(28, 28, 1)

In [18]:
# one hot encode target values
trainY = to_categorical(trainY)
testY = to_categorical(testY)

In [20]:
# confirm scale of pixels
print('Train min=%.3f, max=%.3f' % (trainX.min(), trainX.max()))
print('Test min=%.3f, max=%.3f' % (testX.min(), testX.max()))

Train min=0.000, max=255.000
Test min=0.000, max=255.000


The three main types of pixel scaling techniques supported by the ImageDataGenerator class are as follows:

    Pixel Normalization: scale pixel values to the range 0-1.
    Pixel Centering: scale pixel values to have a zero mean.
    Pixel Standardization: scale pixel values to have a zero mean and unit variance.

The pixel standardization is supported at two levels: either per-image (called sample-wise) or per-dataset (called feature-wise). Specifically, the mean and/or mean and standard deviation statistics required to standardize pixel values can be calculated from the pixel values in each image only (sample-wise) or across the entire training dataset (feature-wise).

Other pixel scaling methods are supported, such as ZCA, brightening, and more, but wel will focus on these three most common methods.

In [21]:
from keras.preprocessing.image import ImageDataGenerator
# create generator (1.0/255.0 = 0.003921568627451)
datagen = ImageDataGenerator(rescale=1.0/255.0)

# prepare an iterators to scale images
train_iterator = datagen.flow(trainX, trainY, batch_size=64)
test_iterator = datagen.flow(testX, testY, batch_size=64)

In [22]:
print('Batches train=%d, test=%d' % (len(train_iterator), len(test_iterator)))

Batches train=938, test=157


In [39]:
# confirm the scaling works
batchX, batchy = train_iterator.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))

Batch shape=(64, 28, 28, 1), min=0.000, max=1.000


In [24]:
# define model 
model = Sequential([
    Conv2D(32, (3,3), activation= 'relu', input_shape = (width, height, channels)), 
    MaxPooling2D((2,2)), 
    Conv2D(64, (3,3), activation= 'relu'), 
    MaxPooling2D((2,2)), 
    Flatten(), 
    Dense(64, activation= 'relu'), 
    Dense(10, activation= 'softmax')
])

In [32]:
# compile model 
adam = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
#mae = tf.keras.metrics.mae()
model.compile(optimizer= adam, loss= 'categorical_crossentropy', metrics= ['accuracy'])

# fit the model 
model.fit_generator(train_iterator, steps_per_epoch= len(train_iterator), epochs= 5)

# evaluate the model 
loss, accuracy = model.evaluate_generator(test_iterator, steps=len(test_iterator), verbose=0)

Epoch 1/5
938/938 [==============================] - 24s 26ms/step - loss: 0.0088 - accuracy: 0.9972
Epoch 2/5
938/938 [==============================] - 29s 31ms/step - loss: 0.0068 - accuracy: 0.9977
Epoch 3/5
938/938 [==============================] - 29s 31ms/step - loss: 0.0058 - accuracy: 0.9980
Epoch 4/5
938/938 [==============================] - 29s 31ms/step - loss: 0.0061 - accuracy: 0.9978
Epoch 5/5
938/938 [==============================] - 29s 31ms/step - loss: 0.0041 - accuracy: 0.9987


In [ ]:
print('Test Accuracy: %.3f' % (accuracy * 100))

Centering the image with ImageDataGenerator()

In [34]:
# create generator that centers pixel values
datagen = ImageDataGenerator(featurewise_center=True)
# calculate the mean on the training dataset
datagen.fit(trainX)

In [37]:
print(datagen.mean)

[[[33.318447]]]


In [45]:
train_iterator = datagen.flow(trainX, trainY, batch_size=64)
test_iterator = datagen.flow(testX, testY, batch_size=64)

In [46]:
# get a batch
batchX, batchy = train_iterator.next()
# mean pixel value in the batch
print(batchX.shape, batchX.mean())
print('Batches train=%d, test=%d' % (len(train_iterator), len(test_iterator)))

(64, 28, 28, 1) 1.3170549
Batches train=938, test=157


In [47]:
# define model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(width, height, channels)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))
# compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# fit model with generator
model.fit_generator(train_iterator, steps_per_epoch=len(train_iterator), epochs=5)
# evaluate model
_, acc = model.evaluate_generator(test_iterator, steps=len(test_iterator), verbose=0)
print('Test Accuracy: %.3f' % (acc * 100))

Epoch 1/5
938/938 [==============================] - 26s 28ms/step - loss: 0.4938 - accuracy: 0.9275
Epoch 2/5
938/938 [==============================] - 28s 29ms/step - loss: 0.0744 - accuracy: 0.9779
Epoch 3/5
938/938 [==============================] - 29s 31ms/step - loss: 0.0510 - accuracy: 0.9847
Epoch 4/5
938/938 [==============================] - 29s 31ms/step - loss: 0.0447 - accuracy: 0.9863
Epoch 5/5
938/938 [==============================] - 29s 31ms/step - loss: 0.0360 - accuracy: 0.98900s - loss: 0.0361 - accuracy: 
Test Accuracy: 98.050


Standardize Image  

In [48]:
# create generator that centers pixel values
datagen = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True)
# calculate the mean on the training dataset
datagen.fit(trainX)

In [49]:
# prepare an iterators to scale images
train_iterator = datagen.flow(trainX, trainY, batch_size=64)
test_iterator = datagen.flow(testX, testY, batch_size=64)
print('Batches train=%d, test=%d' % (len(train_iterator), len(test_iterator)))

Batches train=938, test=157


In [50]:
# define model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(width, height, channels)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))
# compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# fit model with generator
model.fit_generator(train_iterator, steps_per_epoch=len(train_iterator), epochs=5)
# evaluate model
_, acc = model.evaluate_generator(test_iterator, steps=len(test_iterator), verbose=0)
print('Test Accuracy: %.3f' % (acc * 100))

Epoch 1/5
938/938 [==============================] - 25s 27ms/step - loss: 0.1567 - accuracy: 0.9530
Epoch 2/5
938/938 [==============================] - 27s 29ms/step - loss: 0.0487 - accuracy: 0.9851
Epoch 3/5
938/938 [==============================] - 29s 31ms/step - loss: 0.0343 - accuracy: 0.9897
Epoch 4/5
938/938 [==============================] - 29s 31ms/step - loss: 0.0263 - accuracy: 0.9915
Epoch 5/5
938/938 [==============================] - 28s 30ms/step - loss: 0.0197 - accuracy: 0.9937
Test Accuracy: 98.900
